In [63]:
import urllib.request  # импортируем модуль
import re
import urllib.parse
import os
import time
import requests
from requests.exceptions import HTTPError
# from mystem import analyze

In [64]:
def take_Meta(html, url):  # извлекает мета-информацию
    meta = {}
    dating = ''
    pub = ''
    # ищет нужные данные для метаинформаци
    regPostTitle = re.compile('h2 class="itemTitle">(.*?)</h2>',
                                flags= re.DOTALL)
    date = re.compile('<span class="itemDateCreated">.*, (\d* [А-Яа-я]* 20.*) \d.*',
                                flags= re.DOTALL)
    authorship = re.compile('<a rel="author" href=".*">(.*?)</a>.*<div class="itemBody">',
                                flags= re.DOTALL)
    item = re.compile('class="itemCategory">.*<span>Опубликовано в</span>.*<a href="/news/[A-Za-z]*">(.*?)</a>',
                                flags= re.DOTALL)
    author = authorship.search(html)
    titles = regPostTitle.search(html)
    title = titles[1].replace('\n\t\t\t\n\t  \t', '').replace('\n\t  \t\n\t  ', '')
    fin_date = date.search(html)
    category = item.search(html)
    # переводим дату создания в адекватный вид
    time_pub = fin_date[1].split(' ')
    if time_pub[1] != '':
        if time_pub[1] == "Январь":
            pub = "01"
        if time_pub[1] == "Февраль":
            pub = "02"
        if time_pub[1] == "Март":
            pub = "03"
        if time_pub[1] == "Апрель":
            pub = "04"
        if time_pub[1] == "Май":
            pub = "05"
        if time_pub[1] == "Июнь":
            pub = "06"
        if time_pub[1] == "Июль":
            pub = "07"
        if time_pub[1] == "Август":
            pub = "08"
        if time_pub[1] == "Сентябрь":
            pub = "09"
        if time_pub[1] == "Октябрь":
            pub = "10"
        if time_pub[1] == "Ноябрь":
            pub = "11"
        if time_pub[1] == "Декабрь":
            pub = "11"
    dating = str(time_pub[0]) + "." + str(pub) + '.' + str(time_pub[2])
    # создает словарь для бущуего удобного вытаскивания информаци
    meta[title] = [author[1], dating, category[1], url, time_pub[2]]
    print(meta)
    return meta

In [66]:
def article_Text(html):  # функция чистит текст статьи
    # чистит текст
    finding_text = re.compile(
        '<div class="itemBody">.*<!-- Item introtext -->(.*)<!-- Item Rating -->', 
                            flags=re.DOTALL)
    text = finding_text.search(html)
    final_t = str(text[1])
    regTag = re.compile('<.*?>', re.DOTALL)  # рег. выражение находит все тэги
    regScript = re.compile('<script>.*?</script>', re.DOTALL)  # все скрипты
    regComment = re.compile('<!--.*?-->', re.DOTALL)  # все комментарии
    specsymbols_1 = re.compile('&[A-Za-z]*;', re.DOTALL)  # специальные символы
    # а дальше заменяем ненужные куски на пустую строку
    clean_t = regScript.sub("", final_t)
    clean_t = regComment.sub("", clean_t)
    clean_t = regTag.sub("", clean_t)
    clean_text = specsymbols_1.sub("", clean_t)
    return clean_text

In [67]:
def textnTags(clean_text, meta, number):  # создает файл с текстом
    text = str(clean_text)
    tags = ''
    # вытаскивает данные из словаря
    for key, value in meta.items():
        ti = key
        au = value[0]
        da = value[1]
        topic = value[2]
        url = value[3]
        year = value[4]
    dates = da.split('.')
    mounth = dates[1]
    # создает название файла
    filename = "article_" + str(number) + '.txt'
    # делает теги для записи в начале файла
    tags = "@au " + au + "\n@ti " + ti + "\n@da " + \
            da + "\n@topic " + topic + "\n@url " + url
    # создает название папки
    dirname = "plain"
    # прописвает пути
    path_dirname = "%s" % dirname
    if not os.path.exists(path_dirname):
        os.makedirs(path_dirname)
    dirname_year = "%s" % year
    path_year = os.path.join(path_dirname, dirname_year)
    if not os.path.exists(path_year):
        os.makedirs(path_year)
    dirname_mounth = "%s" % mounth
    path_mounth = os.path.join(path_year, dirname_mounth)
    if not os.path.exists(path_mounth):
        os.makedirs(path_mounth)
    # прописывает путь к файлу
    filepath = os.path.join(path_mounth, filename).encode('ascii')
    # создает файл
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(tags)
    with open(filepath, 'a', encoding="utf-8") as file:
        clean = re.sub("  ", "", text)
        file.write(clean)
    return filepath

In [68]:
def creating_Csv(filepath, meta):  # дополняет цсв-файл
    row = ''
    path = str(filepath)
    # достает информацию из словаря для записи в цсв
    for key, value in meta.items():
        author = value[0]
        header = key
        created = value[1]
        topic = value[2]
        source = value[3]
        year = value[4]
    # строка для цсв
    row = '%s\t%s\t%s\t%s\tпублицистика\t%s\tнейтральный\tн-возраст\tн-уровень\tрайонная\t%s\tВолжская правда\t%s\tгазета\tРоссия\tВолжский\tru\n'
    # запись в цсв-файл
    with open('metadata.csv', 'a+', encoding='utf-8') as file:
        file.write(row % (path, author, header, created, topic, source, year))
    return

In [69]:
def mystem(meta, number, filepath):  # создает файлы в mystem
    # достает инофрамацию из словаря для прописывания пути
    for key, value in meta.items():
        da = value[1]
        year = value[4]
    dates = da.split('.')
    mounth = dates[1]
    # имя файла
    filename = "article_" + str(number) + '.txt'
    # прописывает пути для 2-ух форматов майстема
    dirname_2 = "mystem-xml"
    dirname_3 = "mystem-plain"
    path_dirn2 = "%s" % dirname_2
    path_dirn3 = "%s" % dirname_3
    if not os.path.exists(path_dirn2):
        os.makedirs(path_dirn2)
    if not os.path.exists(path_dirn3):
        os.makedirs(path_dirn3)
    dirname_year = "%s" % year
    path_year2 = os.path.join(path_dirn2, dirname_year)
    path_year3 = os.path.join(path_dirn3, dirname_year)
    if not os.path.exists(path_year2):
        os.makedirs(path_year2)
    if not os.path.exists(path_year3):
        os.makedirs(path_year3)
    dirname_mounth = "%s" % mounth
    path_moun2 = os.path.join(path_year2, dirname_mounth)
    path_moun3 = os.path.join(path_year3, dirname_mounth)
    if not os.path.exists(path_moun2):
        os.makedirs(path_moun2)
    if not os.path.exists(path_moun3):
        os.makedirs(path_moun3)
    # пути файлов
    filepath2 = os.path.join(path_moun2, filename).encode('ascii')
    filepath3 = os.path.join(path_moun3, filename).encode('ascii')
    # записывает в файлы информацию
    with open(filepath2, 'w') as f:
        f.write("инфа")
    with open(filepath3, 'w') as f:
        f.write("инфа")
    return

In [70]:
def main():  # запускающая функция
    row = '%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'
    # создает цсв-файл
    with open('metadata.csv', 'w', encoding='utf-8') as file:
        file.write(row % ("path", "author", "header",
                          "created", "sphere", "topic", "style",
                          "audience_age", "audience_level", "audience_size",
                          "source", "publication", "publ_year", "medium",
                          "country", "region", "language"))
    # подает ссылку к статье
    commonUrl = 'http://gazeta-vp.ru/news/./item/'
    for i in range(16943, 17950):  # ищет статьи в диапозоне
        pageUrl = commonUrl + str(i)
        try:
            r = requests.get(pageUrl)
            r.raise_for_status()
        except HTTPError:  # если выдается ошибка 404 - не читается старница
            print (r.url, 'Could not download page')
        else:  # основаня работа с данными
            print (r.url, 'downloaded successfully')
            url = r.url
            number = i
            # притворяемся браузером
            user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
            req = urllib.request.Request(url, headers={'User-Agent':user_agent})
            with urllib.request.urlopen(req) as response:
                html = response.read().decode('utf-8')
            take = take_Meta(html, url)
            text = article_Text(html)
            text_with_tg = textnTags(text, take, number)
            csv = creating_Csv(text_with_tg, take)
            stem = mystem(take, number, text_with_tg)
main()

http://gazeta-vp.ru/news/./item/16943 downloaded successfully
{'В Краснооктябрьском открыли «Дом здоровья»': ['Жанна Лeтяева', '31.07.2018', 'Здравоохранение', 'http://gazeta-vp.ru/news/./item/16943', '2018']}
Название статьи: В Краснооктябрьском открыли «Дом здоровья»
Автор: Жанна Лeтяева
Дата: 31.07.2018
Категория: Здравоохранение
URL: http://gazeta-vp.ru/news/./item/16943
Год публикации: 2018
http://gazeta-vp.ru/news/./item/16944 downloaded successfully
{'«Добровольцам России» дадут 15 миллионов': ['Елена Дунюшкина', '31.07.2018', 'Общество', 'http://gazeta-vp.ru/news/./item/16944', '2018']}
Название статьи: «Добровольцам России» дадут 15 миллионов
Автор: Елена Дунюшкина
Дата: 31.07.2018
Категория: Общество
URL: http://gazeta-vp.ru/news/./item/16944
Год публикации: 2018
http://gazeta-vp.ru/news/./item/16945 downloaded successfully
{'Легко ли оформить маткапитал на детские нужды?': ['Елена Дунюшкина', '31.07.2018', 'Общество', 'http://gazeta-vp.ru/news/./item/16945', '2018']}
Названи

http://gazeta-vp.ru/news/./item/16963 downloaded successfully


KeyboardInterrupt: 